In [45]:
pip install pandas openpyxl yfinance requests


Note: you may need to restart the kernel to use updated packages.


In [48]:
import pandas as pd
import yfinance as yf
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from openpyxl import load_workbook
from openpyxl.chart import PieChart, Reference

file_path = "FinanceTracker.xlsx"
pie_symbols = {"AMZN", "AAPL", "SBUX", "NVDA", "GOOGL"}

# 1. Excel dosyasından "MyStocks" sayfasını oku, başlık 2. satırda (header=1)
try:
    df = pd.read_excel(file_path, sheet_name="MyStocks", header=1)
    print(f"[BİLGİ] Excel dosyasından {len(df)} satır okundu.")
except Exception as e:
    print(f"[HATA] Excel dosyası okunamadı: {e}")
    exit(1)

# 2. Geçersiz ve boş satırları temizle
df = df[~df["Stock Names"].isin(["EXAMPLE", None, pd.NA, ""])]
df = df.dropna(subset=["Stock Names", "Stock Purchase Time", "Stock Sale Time"])
print(f"[BİLGİ] Temizlik sonrası {len(df)} satır kaldı.")

if df.empty:
    print("[UYARI] Veri yok. İşlem sonlandırılıyor.")
    exit(0)

# 3. Sonuçları tutacak liste
data_records = []

# 4. Döngü ile her satırı işle
for index, row in df.iterrows():
    stock_name = row["Stock Names"]
    purchase_time = row["Stock Purchase Time"]
    sale_time = row["Stock Sale Time"]

    print(f"\n[{index+2}. Satır] İşleniyor: Hisse={stock_name}, Alış={purchase_time}, Satış={sale_time}")

        
    # Tarihleri datetime tipine çevir
    try:
        purchase_time = pd.to_datetime(purchase_time)
        sale_time = pd.to_datetime(sale_time)
    except Exception as e:
        print(f"[HATA] Tarih formatı hatası: {e}")
        data_records.append({
            "Stock Purchase Price ($)": None,
            "Stock Sale Price ($)": None,
            "Time Period": None
        })
        continue

    # Yfinance ile alış fiyatı çek
    try:
        ticker = yf.Ticker(stock_name)
        hist_buy = ticker.history(
            start=purchase_time.strftime("%Y-%m-%d"),
            end=(purchase_time + timedelta(days=1)).strftime("%Y-%m-%d"),
            interval="1d"
        )
        if not hist_buy.empty:
            stock_price_buy = round(hist_buy["Close"].iloc[0], 2)
            print(f"[BİLGİ] Alış fiyatı: {stock_price_buy} $")
        else:
            print(f"[UYARI] Alış fiyatı verisi bulunamadı.")
            stock_price_buy = "LIMIT ERROR"
    except Exception as e:
        err_msg = str(e).lower()
        if "rate limit" in err_msg or "limit" in err_msg:
            print(f"[LIMIT HATASI] yfinance limit aşıldı.")
            stock_price_buy = "LIMIT ERROR"
        else:
            print(f"[HATA] Alış fiyatı alınamadı: {e}")
            stock_price_buy = None

    # Yfinance ile satış fiyatı çek
    try:
        hist_sell = ticker.history(
            start=sale_time.strftime("%Y-%m-%d"),
            end=(sale_time + timedelta(days=1)).strftime("%Y-%m-%d"),
            interval="1d"
        )
        if not hist_sell.empty:
            stock_price_sell = round(hist_sell["Close"].iloc[0], 2)
            print(f"[BİLGİ] Satış fiyatı: {stock_price_sell} $")
        else:
            print(f"[UYARI] Satış fiyatı verisi bulunamadı.")
            stock_price_sell = "LIMIT ERROR"
    except Exception as e:
        err_msg = str(e).lower()
        if "rate limit" in err_msg or "limit" in err_msg:
            print(f"[LIMIT HATASI] yfinance limit aşıldı.")
            stock_price_sell = "LIMIT ERROR"
        else:
            print(f"[HATA] Satış fiyatı alınamadı: {e}")
            stock_price_sell = None

    # Zaman farkını hesapla
    try:
        diff = relativedelta(sale_time, purchase_time)
        time_period = f"{diff.years} yıl, {diff.months} ay, {diff.days} gün"
        print(f"[BİLGİ] Zaman farkı: {time_period}")
    except Exception as e:
        print(f"[HATA] Zaman farkı hesaplanamadı: {e}")
        time_period = None

    # Sonuçları listeye ekle
    data_records.append({
        "Stock Purchase Price ($)": stock_price_buy,
        "Stock Sale Price ($)": stock_price_sell,
        "Time Period": time_period
    })

# 5. DataFrame’e çevir
df_result = pd.DataFrame(data_records)
print(f"\n[BİLGİ] Toplam {len(df_result)} satır sonuç oluşturuldu.")
print(df_result.head())

if df_result.empty:
    print("[UYARI] Sonuç DataFrame boş, dosyaya yazma işlemi yapılmayacak.")
else:
    # 6. Excel'e yaz, varsa "DATA" sayfasını değiştir
    try:
        with pd.ExcelWriter(file_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
            df_result.to_excel(writer, sheet_name="DATA", index=False)
        print(" İşlem tamamlandı. Sonuçlar 'DATA' sayfasına yazıldı.")
    except Exception as e:
        print(f"[HATA] Excel dosyasına yazılamadı: {e}")


$AAPL: possibly delisted; no price data found  (1d 2024-06-01 -> 2024-06-02)


[BİLGİ] Excel dosyasından 6 satır okundu.
[BİLGİ] Temizlik sonrası 3 satır kaldı.

[3. Satır] İşleniyor: Hisse=AAPL, Alış=2024-05-01 15:00:00, Satış=2024-06-01 15:00:00
[BİLGİ] Alış fiyatı: 168.28 $
[UYARI] Satış fiyatı verisi bulunamadı.
[BİLGİ] Zaman farkı: 0 yıl, 1 ay, 0 gün

[4. Satır] İşleniyor: Hisse=AMZN, Alış=2024-05-01 15:00:00, Satış=2024-06-01 15:00:00


$AMZN: possibly delisted; no price data found  (1d 2024-06-01 -> 2024-06-02)


[BİLGİ] Alış fiyatı: 179.0 $
[UYARI] Satış fiyatı verisi bulunamadı.
[BİLGİ] Zaman farkı: 0 yıl, 1 ay, 0 gün

[5. Satır] İşleniyor: Hisse=NVDA, Alış=2024-05-01 15:00:00, Satış=2024-06-01 15:00:00


$NVDA: possibly delisted; no price data found  (1d 2024-06-01 -> 2024-06-02)


[BİLGİ] Alış fiyatı: 83.01 $
[UYARI] Satış fiyatı verisi bulunamadı.
[BİLGİ] Zaman farkı: 0 yıl, 1 ay, 0 gün

[BİLGİ] Toplam 3 satır sonuç oluşturuldu.
   Stock Purchase Price ($) Stock Sale Price ($)         Time Period
0                    168.28          LIMIT ERROR  0 yıl, 1 ay, 0 gün
1                    179.00          LIMIT ERROR  0 yıl, 1 ay, 0 gün
2                     83.01          LIMIT ERROR  0 yıl, 1 ay, 0 gün
 İşlem tamamlandı. Sonuçlar 'DATA' sayfasına yazıldı.


In [49]:
import pandas as pd
import yfinance as yf
from openpyxl import load_workbook
from openpyxl.chart import PieChart, Reference
from datetime import timedelta, datetime
from dateutil.relativedelta import relativedelta

# Excel dosya yolu
file_path = "FinanceTracker.xlsx"

# İzlenecek hisseler
symbols = ["AMZN", "AAPL", "SBUX", "NVDA", "GOOGL"]

# Tarih bilgileri
purchase_time = pd.to_datetime("2024-05-01")
sale_time = pd.to_datetime(datetime.today().date())

# Fiyat verisini tek seferde çek
data = yf.download(
    tickers=symbols,
    start=purchase_time.strftime("%Y-%m-%d"),
    end=(sale_time + timedelta(days=1)).strftime("%Y-%m-%d"),
    interval="1d"
)

# Verileri toplamak için listeler
data_records = []
pie_data = []

for stock in symbols:
    try:
        hist = data["Close"][stock]
        price_buy = hist.loc[str(purchase_time.date())] if str(purchase_time.date()) in hist else None
        price_sell = hist.iloc[-1] if not hist.empty else None

        if price_buy is not None and price_sell is not None:
            price_buy = round(price_buy, 2)
            price_sell = round(price_sell, 2)
            diff = relativedelta(sale_time, purchase_time)
            time_period = f"{diff.years} yıl, {diff.months} ay, {diff.days} gün"

            data_records.append({
                "Stock Names": stock,
                "Stock Purchase Price ($)": price_buy,
                "Stock Sale Price ($)": price_sell,
                "Time Period": time_period
            })

            pie_data.append((stock, price_sell))
        else:
            print(f"[UYARI] {stock} için veri eksik.")
    except Exception as e:
        print(f"[HATA] {stock} verisi alınamadı: {e}")

# Sonuçları DataFrame’e çevir
df_result = pd.DataFrame(data_records)

# Excel'e yaz
with pd.ExcelWriter(file_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df_result.to_excel(writer, sheet_name="DATA", index=False)

# Pie chart oluştur
wb = load_workbook(file_path)
if "PieLive" in wb.sheetnames:
    del wb["PieLive"]
ws = wb.create_sheet("PieLive")

# Başlıklar ve veri
ws["A1"] = "Stock"
ws["B1"] = "Last Price"
for i, (symbol, price) in enumerate(pie_data, start=2):
    ws[f"A{i}"] = symbol
    ws[f"B{i}"] = price

chart = PieChart()
chart.title = "Güncel Hisse Dağılımı"
data_range = Reference(ws, min_col=2, min_row=1, max_row=len(pie_data)+1)
label_range = Reference(ws, min_col=1, min_row=2, max_row=len(pie_data)+1)
chart.add_data(data_range, titles_from_data=True)
chart.set_categories(label_range)
ws.add_chart(chart, "D2")

wb.save(file_path)
print("✅ Excel başarıyla güncellendi ve pie chart oluşturuldu.")


[*********************100%***********************]  5 of 5 completed

✅ Excel başarıyla güncellendi ve pie chart oluşturuldu.
